<a href="https://colab.research.google.com/github/rachelburton/TEA_Electrolyzer/blob/main/Steam_Electrolyzer_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Create Steam Electrolyzer Plot

### Import Data & Cross-Join all combinations of Current & Resistance
- shows all combinations of current & resistance

In [32]:
import pandas as pd
import numpy as np
from itertools import product
!pip install openpyxl

In [33]:
# Read the Excel file into separate dataframes
file_name = 'CoLab Inputs - Steam Electrolyzer.xlsx'

#future versions: Make this loop through the tabs
current_density_df = pd.read_excel(file_name, sheet_name='CurrentDensity', header=0)
resistance_df = pd.read_excel(file_name, sheet_name='Resistance', header=0)

#crossjoin dataframes to create an instance for every I & R combo
current_density_df['_tmpkey'] = 1
resistance_df['_tmpkey'] = 1

#create cross-join and drop temp keys
df_cross = pd.merge(current_density_df, resistance_df, on='_tmpkey').drop('_tmpkey',axis=1)

# Add an indexing column to df - need this to be able to call out specific reference architectures later
df_cross.reset_index(inplace=True)

# Rename the new index column:
df_cross.rename(columns={'index': 'Electrolyzer Index'}, inplace=True)

print(df_cross)

"""
# Iterate through each sheet
for sheet_name in workbook.sheet_names:
  # Read data into a pandas DataFrame
  temp =  pd.read_excel(file_name, sheet_name=sheet_name, header=0)
  temp.insert(0, 'Attribute', sheet_name)
"""

    Electrolyzer Index  Current Density [A/cm^2]  Resistance [ohm-cm^2]
0                    0                       0.3                    1.0
1                    1                       0.3                    0.8
2                    2                       0.3                    0.7
3                    3                       0.3                    0.6
4                    4                       0.3                    0.5
..                 ...                       ...                    ...
58                  58                       3.0                    0.5
59                  59                       3.0                    0.4
60                  60                       3.0                    0.3
61                  61                       3.0                    0.2
62                  62                       3.0                    0.1

[63 rows x 3 columns]


"\n# Iterate through each sheet\nfor sheet_name in workbook.sheet_names:\n  # Read data into a pandas DataFrame\n  temp =  pd.read_excel(file_name, sheet_name=sheet_name, header=0)\n  temp.insert(0, 'Attribute', sheet_name)\n"

### Calculate OPEX Values
- Assume no auxiliary process OPEX energy costs (i.e. don't account for water purification, water heating (assume can get waste heat, H2 drying & H2 compression)

In [34]:
#create resistance voltage dictionary
#maps current density to it's respective voltage correction amount (V), first column is current density, second is voltage correction

voltage_correction_dict = {0.3: 1.5,
                           0.5: 1.52,
                           1: 1.6,
                           1.5: 1.65,
                           2: 1.75,
                           2.5: 1.9,
                           3: 2.2
                           }
#print(dir(voltage_correction_dict))
#print(help(voltage_correction_dict))
print(voltage_correction_dict.get(1.5))


1.65


In [35]:
#Constants
electrons_per_amp = 6.28*10**18
electrons_per_H2molecule = 2
H2_mols_per_molecule = 6.02*10**23
kgH2_per_molsH2 = 0.00202

def calc_kgH2perHour(current_density, electrons_per_amp, electrons_per_H2molecule,kgH2_per_molsH2,H2_mols_per_molecule, ):
    electrons_per_second = current_density*electrons_per_amp
    H2molecuule_per_second = electrons_per_second/electrons_per_H2molecule
    kgH2perHour = ((H2molecuule_per_second*3600)/H2_mols_per_molecule)*kgH2_per_molsH2
    return kgH2perHour

def calc_hours_per_kgH2(kgH2perHour):
    hours_per_kgH2 = 1/kgH2perHour
    return hours_per_kgH2

def calc_hours_per_kgH2(kgH2perHour):
    hours_per_kgH2 = 1/kgH2perHour
    return hours_per_kgH2

def calc_Stack_energy_consumption(hours_per_kgH2,power_kW):
    stack_energy_consumption = hours_per_kgH2*power_kW
    return stack_energy_consumption

In [36]:
#Create formulas & populate df
df_cross['Voltage [V]'] = df_cross['Current Density [A/cm^2]'] * df_cross['Resistance [ohm-cm^2]']
df_cross['Voltage Actual [V]'] = df_cross.apply(lambda row: row['Voltage [V]'] + voltage_correction_dict.get(row['Current Density [A/cm^2]'], 0), axis=1)
df_cross['Power [kW]'] = df_cross['Current Density [A/cm^2]'] * df_cross['Voltage Actual [V]']/1000
df_cross['kg H2/hour'] = df_cross.apply(lambda row: calc_kgH2perHour(row['Current Density [A/cm^2]'], electrons_per_amp, electrons_per_H2molecule, kgH2_per_molsH2,H2_mols_per_molecule), axis=1)
df_cross['Hours/kg H2'] = df_cross.apply(lambda row: calc_hours_per_kgH2(row['kg H2/hour']), axis=1)
df_cross['Stack Energy Consumption [kWh/kg]'] = df_cross.apply(lambda row: calc_Stack_energy_consumption(row['Hours/kg H2'],row['Power [kW]']), axis=1)
#added in a random CAPEX value to test plotting...will come back & correct
df_cross['CAPEX_Random_Number [$1000]'] = np.random.randint(800,1200, size=len(df_cross))
df_cross['Raw Material Avg Cost per KG [$]'] = 30
df_cross['Stack Cost [$/kW]']= 896 #update if this changes from India
df_cross['Stack Cost at 10MW [$]'] = df_cross['Stack Cost [$/kW]'] * 1000 * 10 #kw to MW then to 10 MW
print(df_cross)


    Electrolyzer Index  Current Density [A/cm^2]  Resistance [ohm-cm^2]  \
0                    0                       0.3                    1.0   
1                    1                       0.3                    0.8   
2                    2                       0.3                    0.7   
3                    3                       0.3                    0.6   
4                    4                       0.3                    0.5   
..                 ...                       ...                    ...   
58                  58                       3.0                    0.5   
59                  59                       3.0                    0.4   
60                  60                       3.0                    0.3   
61                  61                       3.0                    0.2   
62                  62                       3.0                    0.1   

    Voltage [V]  Voltage Actual [V]  Power [kW]  kg H2/hour   Hours/kg H2  \
0          0.30       

In [37]:
#filter DF for combinations that are below thermodynamic limit (check with India)
# Filter df_cross to remove options below 30 kWh/kg
df_cross = df_cross[df_cross['Stack Energy Consumption [kWh/kg]'] >= 30]
print(df_cross)

    Electrolyzer Index  Current Density [A/cm^2]  Resistance [ohm-cm^2]  \
0                    0                       0.3                    1.0   
1                    1                       0.3                    0.8   
2                    2                       0.3                    0.7   
3                    3                       0.3                    0.6   
4                    4                       0.3                    0.5   
..                 ...                       ...                    ...   
58                  58                       3.0                    0.5   
59                  59                       3.0                    0.4   
60                  60                       3.0                    0.3   
61                  61                       3.0                    0.2   
62                  62                       3.0                    0.1   

    Voltage [V]  Voltage Actual [V]  Power [kW]  kg H2/hour   Hours/kg H2  \
0          0.30       

##Add in competitor data

In [38]:
#Read in competitor Data & create updated df with internal & competitor data
file_name = 'External_Data.xlsx'
sheet_name='Sheet1'

def create_df_from_excel(file_name, sheet_name):
    try:
      df_competitor = pd.read_excel(file_name, sheet_name=sheet_name)  # engine parameter has been deprecated for read_excel
      print(f"DataFrame created from '{file_name}' successfully.")
      return df_competitor
    except Exception as e:
      print(f"An error occurred while creating the DataFrame: {e}")
      return None


df_competitor = create_df_from_excel(file_name, sheet_name)
if df_competitor is not None:
    print(df_competitor)

DataFrame created from 'External_Data.xlsx' successfully.
                         Electrolyzer Index  \
0                           Achieved_In_Lab   
1                 PEM: Plug EX-2125D - 5 MW   
2                            PEM: NEL MC250   
3                            PEM: NEL MC500   
4     PEM: Cummins HyLYZER 200 - optimistic   
5   PEM: Cummins HyLYZER 200 - conservative   
6     PEM: Cummins HyLYZER 250 - optimistic   
7   PEM: Cummins HyLYZER 250 - conservative   
8       PEM: Electric Hydrogen - optimistic   
9       PEM: Electric Hydrogen - pesimistic   
10        PEM: Advanced Ionics - optimistic   
11        PEM: Advanced Ionics - pesimistic   
12           Alkaline: Longi Alki H1 Series   
13        Alkaline: Sunfire-HyLink Alkaline   
14            Alkaline: Thyssen Krupp 20 MW   

                   Current Density [A/cm^2]  \
0                           Achieved_In_Lab   
1                 PEM: Plug EX-2125D - 5 MW   
2                            PEM: NEL MC250   
3

In [39]:
#create updated dataframe with internal and competitor data
df1 = df_cross
df2 = df_competitor

combined_df = pd.concat([df1, df2], ignore_index=True)
print(combined_df)
combined_df.to_csv('combined_data.csv', index=False)

                   Electrolyzer Index           Current Density [A/cm^2]  \
0                                   0                                0.3   
1                                   1                                0.3   
2                                   2                                0.3   
3                                   3                                0.3   
4                                   4                                0.3   
..                                ...                                ...   
73  PEM: Advanced Ionics - optimistic  PEM: Advanced Ionics - optimistic   
74  PEM: Advanced Ionics - pesimistic  PEM: Advanced Ionics - pesimistic   
75     Alkaline: Longi Alki H1 Series     Alkaline: Longi Alki H1 Series   
76  Alkaline: Sunfire-HyLink Alkaline  Alkaline: Sunfire-HyLink Alkaline   
77      Alkaline: Thyssen Krupp 20 MW      Alkaline: Thyssen Krupp 20 MW   

                Resistance [ohm-cm^2]                        Voltage [V]  \
0          

## Plot tradespace with competitor data

In [40]:
combined_df

,Electrolyzer Index,Current Density [A/cm^2],Resistance [ohm-cm^2],Voltage [V],Voltage Actual [V],Power [kW],kg H2/hour,Hours/kg H2,Stack Energy Consumption [kWh/kg],CAPEX_Random_Number [$1000],Raw Material Avg Cost per KG [$],Stack Cost [$/kW],Stack Cost at 10MW [$]
0,0,0.3,1.0,0.3,1.8,0.00054,0.000011,87880.337928,47.455382,1143,30,896,8960000.0
1,1,0.3,0.8,0.24,1.74,0.000522,0.000011,87880.337928,45.873536,810,30,896,8960000.0
2,2,0.3,0.7,0.21,1.71,0.000513,0.000011,87880.337928,45.082613,1069,30,896,8960000.0
3,3,0.3,0.6,0.18,1.68,0.000504,0.000011,87880.337928,44.291690,942,30,896,8960000.0
4,4,0.3,0.5,0.15,1.65,0.000495,0.000011,87880.337928,43.500767,841,30,896,8960000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,PEM: Advanced Ionics - optimistic,PEM: Advanced Ionics - optimistic,PEM: Advanced Ionics - optimistic,PEM: Advanced Ionics - optimistic,PEM: Advanced Ionics - optimistic,PEM: Advanced Ionics - optimistic,PEM: Advanced Ionics - optimistic,PEM: Advanced Ionics - optimistic,30.000000,350,200,1038,10388000.0
74,PEM: Advanced Ionics - pesimistic,PEM: Advanced Ionics - pesimistic,PEM: Advanced Ionics - pesimistic,PEM: Advanced Ionics - pesimistic,PEM: Advanced Ionics - pesimistic,PEM: Advanced Ionics - pesimistic,PEM: Advanced Ionics - pesimistic,PEM: Advanced Ionics - pesimistic,35.000000,300,200,1038,10388000.0
75,Alkaline: Longi Alki H1 Series,Alkaline: Longi Alki H1 Series,Alkaline: Longi Alki H1 Series,Alkaline: Longi Alki H1 Series,Alkaline: Longi Alki H1 Series,10000,NaN,NaN,47.816000,300,50,996,9964000.0
76,Alkaline: Sunfire-HyLink Alkaline,Alkaline: Sunfire-HyLink Alkaline,Alkaline: Sunfire-HyLink Alkaline,Alkaline: Sunfire-HyLink Alkaline,Alkaline: Sunfire-HyLink Alkaline,10000,200,0.005,50.151200,300,50,996,9964000.0


###Current Density Comparison

In [44]:
import plotly.graph_objects as go
import matplotlib.cm as cm
import matplotlib.pyplot as plt

# Define your custom color list
my_colors = ['red', 'blue', 'green', 'orange', 'purple', 'brown', 'pink', 'gray', 'black', 'cyan', 'magenta', 'yellow', 'lime', 'teal', 'navy', 'olive', 'maroon', 'aquamarine', 'coral', 'gold']

# Define a function to create a color map based on data type
def create_color_map(unique_labels):
    color_map = {}
    numeric_labels = [label for label in unique_labels if isinstance(label, (int, float))]

    # Create a color map for numeric labels using a built-in colormap
    if numeric_labels:
        norm = plt.Normalize(min(numeric_labels), max(numeric_labels))
        cmap = cm.get_cmap('viridis')  # Choose a colormap (e.g., 'viridis', 'plasma', 'inferno')
        for label in numeric_labels:
            color_map[label] = cmap(norm(label))[:3]  # Get RGB values from colormap

    # Add colors for non-numeric labels (words) using hashing
    non_numeric_labels = [label for label in unique_labels if label not in numeric_labels]
    num_colors = len(non_numeric_labels)  # Get the number of unique non-numeric labels
    cmap = cm.get_cmap('Paired', num_colors)  # Use a cyclic colormap like 'hsv'

    for i, label in enumerate(non_numeric_labels):
        #color_map[label] = cmap(i)[:3]  # Assign a unique color from the colormap
        color_map[label] = my_colors[i % len(my_colors)]  # Cycle through your custom colors

    return color_map

# Get unique labels from your DataFrame
unique_labels = combined_df['Current Density [A/cm^2]'].unique()

# Create the color map
color_map = create_color_map(unique_labels)

"""
color_map = {
    0.3: 'red',
    0.5: 'blue',
    1.0: 'green',
    1.5: 'orange',
    2.0: 'purple',
    2.5: 'yellow',
    'CompanyA': 'black',  # Add color for 'CompanyA'
    'CompanyB': 'cyan',  # Add color for 'CompanyB'
    # ... add more if needed
}
"""
fig = go.Figure()

fig.update_layout(
    height=700,
    width=800,
    title="Stack Energy Consumption per kg vs. Stack Cost at 10MW [$]",
    xaxis_title="Stack Cost at 10MW [$]",
    yaxis_title="Stack Energy Consumption per kg H2 [kWh/kg]",
    legend_title_text="Current Density",
    font=dict(size=14),
    autosize=False,)

# Add customdata to the traces (if not already present)
for i in range(len(fig.data)):
    hovertext_values = combined_df.loc[combined_df['Electrolyzer Index']== fig.data[i].name,'Electrolyzer Index'].tolist()
    fig.data[i].update(
        customdata=combined_df[['Current Density [A/cm^2]', 'Resistance [ohm-cm^2]']],
        hovertemplate="<br>".join([
            "<b>Electrolyzer Index:</b> %{hovertext}",  # Display Electrolyzer Index
            #"<b>CAPEX:</b> %{x:.2f} [$1000]",          # Display CAPEX with 2 decimal places
           #"<b>Energy Consumption:</b> %{y:.2f} [kWh/kg]",  # Display Energy Consumption with 2 decimal places
            "<b>Current Density:</b> %{customdata[0]:.2f} [A/cm^2]", #Current Density
            "<b>Resistance:</b> %{customdata[1]:.2f} [ohm-cm^2]"  #Resistance
        ]),
        hovertext=hovertext_values # Assign hovertext within the loop
    )

for label in combined_df['Current Density [A/cm^2]'].unique():
    # Create a copy of the DataFrame for filtering
    filtered_df = combined_df[combined_df['Current Density [A/cm^2]'] == label].copy()  # Use .copy() to avoid SettingWithCopyWarning
    #filtered_df = combined_df[combined_df['Current Density [A/cm^2]'] == label]

    # Determine marker properties based on label type
    marker_props = dict()
    if isinstance(label, (int, float)):
        marker_props['color'] = color_map.get(label, 'gray')  # Use color_map or default to gray
        marker_props['symbol'] = 'circle'  # Default symbol for numeric labels
        legend_label = f"Current Density: {label} A/cm^2"  # Create legend label with prefix
    else:  # Non-numeric label (string or other)
        marker_props['color'] = color_map.get(label, 'gray')  # Use color_map or default to gray
        marker_props['symbol'] = 'square'  # Use star symbol for non-numeric labels
        marker_props['size'] = 10  # Set size for non-numeric labels
        legend_label = str(label)

    fig.add_trace(go.Scatter(
        x=filtered_df['Stack Cost at 10MW [$]'],
        y=filtered_df['Stack Energy Consumption [kWh/kg]'],
        mode='markers',
        name=legend_label,  # Set the legend label
        marker=marker_props
    ))

# Update layout to enable hover
fig.update_layout(hovermode="closest")
fig.show()



<ipython-input-44-27501e0c628f>:16: MatplotlibDeprecationWarning:

The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.

<ipython-input-44-27501e0c628f>:23: MatplotlibDeprecationWarning:

The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.



###Resistance Comparison

In [46]:
import plotly.graph_objects as go
import matplotlib.cm as cm
import matplotlib.pyplot as plt

# Define your custom color list
my_colors = ['red', 'blue', 'green', 'orange', 'purple', 'brown', 'pink', 'gray', 'black', 'cyan', 'magenta', 'yellow', 'lime', 'teal', 'navy', 'olive', 'maroon', 'aquamarine', 'coral', 'gold']

# Define a function to create a color map based on data type
def create_color_map(unique_labels):
    color_map = {}
    numeric_labels = [label for label in unique_labels if isinstance(label, (int, float))]

    # Create a color map for numeric labels using a built-in colormap
    if numeric_labels:
        norm = plt.Normalize(min(numeric_labels), max(numeric_labels))
        cmap = cm.get_cmap('viridis')  # Choose a colormap (e.g., 'viridis', 'plasma', 'inferno')
        for label in numeric_labels:
            color_map[label] = cmap(norm(label))[:3]  # Get RGB values from colormap

    # Add colors for non-numeric labels (words) using hashing
    non_numeric_labels = [label for label in unique_labels if label not in numeric_labels]
    num_colors = len(non_numeric_labels)  # Get the number of unique non-numeric labels
    cmap = cm.get_cmap('Paired', num_colors)  # Use a cyclic colormap like 'hsv'

    for i, label in enumerate(non_numeric_labels):
        #color_map[label] = cmap(i)[:3]  # Assign a unique color from the colormap
        color_map[label] = my_colors[i % len(my_colors)]  # Cycle through your custom colors

    return color_map

# Get unique labels from your DataFrame
unique_labels = combined_df['Resistance [ohm-cm^2]'].unique()

# Create the color map
color_map = create_color_map(unique_labels)

"""
color_map = {
    0.3: 'red',
    0.5: 'blue',
    1.0: 'green',
    1.5: 'orange',
    2.0: 'purple',
    2.5: 'yellow',
    'CompanyA': 'black',  # Add color for 'CompanyA'
    'CompanyB': 'cyan',  # Add color for 'CompanyB'
    # ... add more if needed
}
"""
fig = go.Figure()

fig.update_layout(
    height=700,
    width=800,
    title="Stack Energy Consumption per kg vs. Stack Cost at 10MW [$]",
    xaxis_title="Stack Cost at 10MW [$]",
    yaxis_title="Stack Energy Consumption per kg H2 [kWh/kg]",
    legend_title_text="Resistance",
    font=dict(size=14),
    autosize=False,)

# Add customdata to the traces (if not already present)
for i in range(len(fig.data)):
    hovertext_values = combined_df.loc[combined_df['Electrolyzer Index']== fig.data[i].name,'Electrolyzer Index'].tolist()
    fig.data[i].update(
        customdata=combined_df[['Current Density [A/cm^2]', 'Resistance [ohm-cm^2]']],
        hovertemplate="<br>".join([
            "<b>Electrolyzer Index:</b> %{hovertext}",  # Display Electrolyzer Index
            #"<b>CAPEX:</b> %{x:.2f} [$1000]",          # Display CAPEX with 2 decimal places
           #"<b>Energy Consumption:</b> %{y:.2f} [kWh/kg]",  # Display Energy Consumption with 2 decimal places
            "<b>Current Density:</b> %{customdata[0]:.2f} [A/cm^2]", #Current Density
            "<b>Resistance:</b> %{customdata[1]:.2f} [ohm-cm^2]"  #Resistance
        ]),
        hovertext=hovertext_values # Assign hovertext within the loop
    )

for label in combined_df['Resistance [ohm-cm^2]'].unique():
    # Create a copy of the DataFrame for filtering
    filtered_df = combined_df[combined_df['Resistance [ohm-cm^2]'] == label].copy()  # Use .copy() to avoid SettingWithCopyWarning
    #filtered_df = combined_df[combined_df['Current Density [A/cm^2]'] == label]

    # Determine marker properties based on label type
    marker_props = dict()
    if isinstance(label, (int, float)):
        marker_props['color'] = color_map.get(label, 'gray')  # Use color_map or default to gray
        marker_props['symbol'] = 'circle'  # Default symbol for numeric labels
        legend_label = f"Resistance: {label} ohm-cm^2"  # Create legend label with prefix
    else:  # Non-numeric label (string or other)
        marker_props['color'] = color_map.get(label, 'gray')  # Use color_map or default to gray
        marker_props['symbol'] = 'square'  # Use star symbol for non-numeric labels
        marker_props['size'] = 10  # Set size for non-numeric labels
        legend_label = str(label)

    fig.add_trace(go.Scatter(
        x=filtered_df['Stack Cost at 10MW [$]'],
        y=filtered_df['Stack Energy Consumption [kWh/kg]'],
        mode='markers',
        name=legend_label,  # Set the legend label
        marker=marker_props
    ))

# Update layout to enable hover
fig.update_layout(hovermode="closest")
fig.show()



<ipython-input-46-f5a1ce235bee>:16: MatplotlibDeprecationWarning:

The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.

<ipython-input-46-f5a1ce235bee>:23: MatplotlibDeprecationWarning:

The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.



In [42]:
import plotly.graph_objects as go

fig = go.Figure()

# Get unique combinations of current density and resistance
unique_combinations = df_cross[['Current Density [A/cm^2]', 'Resistance [ohm-cm^2]']].drop_duplicates()

# Add a trace for each unique combination
for index, row in unique_combinations.iterrows():
    current_density = row['Current Density [A/cm^2]']
    resistance = row['Resistance [ohm-cm^2]']
    filtered_df = df_cross[
        (df_cross['Current Density [A/cm^2]'] == current_density) &
        (df_cross['Resistance [ohm-cm^2]'] == resistance)
    ]

    legend_label = f"Current Density: {current_density}, Resistance: {resistance}"

    fig.add_trace(go.Scatter(
        x=filtered_df['CAPEX_Random_Number [$1000]'],
        y=filtered_df['Stack Energy Consumption [kWh/kg]'],
        mode='markers',
        name=legend_label,  # Set the legend label
        marker=dict(color=current_density) # Optionally set color based on density for visual distinction
    ))

fig.update_layout(showlegend=True)  # Ensure the legend is visible
fig.show()

#Build capability to explore tradespace

In [28]:
#Get the indices for our three original reference architectures
conservative_row_index = df_cross.index[(df_cross['Current Density [A/cm^2]'] == 1.5) &
                              (df_cross['Resistance [ohm-cm^2]'] == 0.5)]
nominal_row_index = df_cross.index[(df_cross['Current Density [A/cm^2]'] == 2) &
                              (df_cross['Resistance [ohm-cm^2]'] == 0.3)]
optimistic_row_index = df_cross.index[(df_cross['Current Density [A/cm^2]'] == 2.5) &
                            (df_cross['Resistance [ohm-cm^2]'] == 0.1)]

#unique_architectures_index = [conservative_row_index, nominal_row_index, optimistic]
unique_architectures_index = conservative_row_index.union(nominal_row_index).union(optimistic_row_index)
unique_architectures = df_cross.loc[unique_architectures_index,'Electrolyzer Index'].tolist() # tolist added to match original list type
#unique_architectures = df_cross.loc[unique_architectures_index,'Electrolyzer Index']

print(conservative_row_index)
print(nominal_row_index)
print(optimistic_row_index)
#print(unique_architectures_index)
print(unique_architectures)

Index([31], dtype='int64')
Index([42], dtype='int64')
Index([53], dtype='int64')
[31, 42, 53]


In [29]:
# Filter the tradespace_df based on the values of the variables
#ref_architecture = df_cross[] #didn't add any filters

ref_architecture = df_cross[
    (df_cross['Current Density [A/cm^2]'] == Current_density) &
    (df_cross['Resistance [ohm-cm^2]'] == Resistance)
]

# Now, 'filtered_tradespace' contains the rows matching the specified criteria.
# You can access the row(s) using indexing or other DataFrame methods.

# For example, to print the first matching row:
if not ref_architecture.empty:
  ref_architecture = ref_architecture.T
else:
  print("No matching architectures found in the tradespace.")

NameError: name 'Current_density' is not defined

#Scatch Notes - Ignore

In [30]:
#Plot by Resistance
import plotly.graph_objects as go

fig = go.Figure()

# Get unique 'Resistance [ohm-cm^2]' values
unique_resistances = df_cross['Resistance [ohm-cm^2]'].unique()

# Add a trace for each unique current resistance
for resistance in unique_resistances:
    filtered_df = df_cross[df_cross['Resistance [ohm-cm^2]'] == resistance]
    fig.add_trace(go.Scatter(
        x=filtered_df['CAPEX_Random_Number [$1000]'],
        y=filtered_df['Stack Energy Consumption [kWh/kg]'],
        mode='markers',
        name=f"Resistance: {resistance} ohm-cm^2",
        marker=dict(color=resistance) # Optionally set color based on resistance for visual distinction
    ))

fig.update_layout(
    title = "Stack Energy Consumption per kg vs. CAPEX [$1000]",
    xaxis_title="CAPEX [$1000]",
    yaxis_title="Stack Energy Consumption [kWh/kg]",
    font=dict(size=14),
    autosize=False,
    width=800,
    height=600,
    showlegend=True)
fig.show()

In [ ]:
#Plot by Current Density
import plotly.graph_objects as go

fig = go.Figure()

# Get unique 'Current Density [A/cm^2]' values
unique_current_densities = df_cross['Current Density [A/cm^2]'].unique()

# Add a trace for each unique current density
for density in unique_current_densities:
    filtered_df = df_cross[df_cross['Current Density [A/cm^2]'] == density]
    fig.add_trace(go.Scatter(
        x=filtered_df['CAPEX_Random_Number [$1000]'],
        y=filtered_df['Stack Energy Consumption [kWh/kg]'],
        mode='markers',
        name=f"Current Density: {density} A/cm^2",  # Set the name for the legend using current density
        marker=dict(color=density) # Optionally set color based on density for visual distinction
    ))

fig.update_layout(
    title = "Stack Energy Consumption per kg vs. CAPEX [$1000]",
    xaxis_title="CAPEX [$1000]",
    yaxis_title="Stack Energy Consumption [kWh/kg]",
    font=dict(size=14),
    autosize=False,
    width=800,
    height=600,
    showlegend=True)
fig.show()

In [ ]:
#Get Reference Architecture
# Road Vehicle Design Options: #Rachel edited for reference architecture
Current_density = 1
Resistance = 0.5

In [ ]:
# #Plot by Current Density with competitors
# import plotly.graph_objects as go

# fig = go.Figure()

# for label in combined_df['Current Density [A/cm^2]'].unique():
#     combined_df = combined_df[combined_df['Current Density [A/cm^2]'] == label]
#     fig.add_trace(go.Scatter(
#         x=combined_df['CAPEX_Random_Number [$1000]'],
#         y=combined_df['Stack Energy Consumption [kWh/kg]'],
#         mode='markers',
#         name=str(label),  # Convert label to string for legend
#         marker=dict(color=label) if isinstance(label, (int, float)) else dict(symbol=label) if isinstance(label, str) else dict() # if label is a number assign to color, if a string assign to symbol else an empty dictionary
#     ))

# fig.update_layout(
#     title="Stack Energy Consumption per kg vs. CAPEX [$1000]",
#     xaxis_title="CAPEX [$1000]",
#     yaxis_title="Stack Energy Consumption [kWh/kg]",
#     font=dict(size=14),
#     autosize=False,
#     width=800,
#     height=600,
#     showlegend=True
# )
# fig.show()

# # # Define a color map (example using a dictionary)
# color_map = {
#     if combined_df['Current Density [A/cm^2]'] == :
#     0.3: 'red',
#     0.5: 'blue',
#     1.0: 'green',
#     1.5: 'orange',
#     2.0: 'purple',
#     2.5: 'yellow', #Continue this dict based on the range of current densities in the dataframe.
# }

# def categorize_density(density):
#     if density < 0.5:
#         return "Low"
#     elif density < 1.0:
#         return "Medium"
#     elif density < 1.5:
#         return "High"
#     else:
#         return "Very High"  # Adjust categories as needed


# # Get unique 'Current Density [A/cm^2]' values
# unique_current_densities = combined_df['Current Density [A/cm^2]'].unique()

# # Add a trace for each unique current density
# for density in unique_current_densities:
#     filtered_df = combined_df[combined_df['Current Density [A/cm^2]'] == density]
#     fig.add_trace(go.Scatter(
#         x=filtered_df['CAPEX_Random_Number [$1000]'],
#         y=filtered_df['Stack Energy Consumption [kWh/kg]'],
#         mode='markers',
#         name=f"Current Density: {density} A/cm^2",  # Set the name for the legend using current density
#         marker=dict(color=color_map[density]) # Optionally set color based on density for visual distinction
#     ))

# fig.update_layout(
#     title = "Stack Energy Consumption per kg vs. CAPEX [$1000]",
#     xaxis_title="CAPEX [$1000]",
#     yaxis_title="Stack Energy Consumption [kWh/kg]",
#     font=dict(size=14),
#     autosize=False,
#     width=800,
#     height=600,
#     showlegend=True)
# fig.show()

In [ ]:
output_fig = plot_tradespace(df_cross, color_by='Current Density [A/cm^2]', filter=None, filter_range=None)#removed ref_architectures=[]as last arg
output_fig.show()

In [ ]:
import plotly.graph_objects as go

#reference architectures can be filled in by the indices above
#need to define reference architectures?
def plot_tradespace(df_cross, color_by=None, filter=None, filter_range=None, ref_architectures=[42]):
    starting_length = len(df_cross)

    """
    # Apply filter if specified
    if filter is not None:
        tradespace_df = tradespace_df[
            (tradespace_df[filter] >= filter_range[0]) &
            (tradespace_df[filter] <= filter_range[1])
        ]
        print(f'Filtered out {starting_length - len(tradespace_df)} architectures')
"""

    # # Separate reference architectures into two different dfs
    ref_df = df_cross[df_cross['Electrolyzer Index'].isin(ref_architectures)]
    non_ref_df = df_cross[~df_cross['Electrolyzer Index'].isin(ref_architectures)]

    # Create main scatter plot for non-reference architectures
    fig = go.Figure()


    # Check if `color_by` is specified
    if color_by:
        # Loop through unique values in the `color_by` column to create separate traces
        for color_value in non_ref_df[color_by].unique():
            subset_df = non_ref_df[non_ref_df[color_by] == color_value]
            fig.add_trace(go.Scatter(
                x=subset_df["CAPEX_Random_Number [$1000]"],
                y=subset_df["Stack Energy Consumption [kWh/kg]"],
                mode='markers',
                marker=dict(
                    size=8,
                    opacity=0.7
                ),
                name=f'{color_by}: {color_value}',  # Add each unique value to the legend
                hovertext=subset_df['Electrolyzer Index']
            ))
    else:
        # If no color_by column is specified, plot all non-reference architectures as a single trace
        fig.add_trace(go.Scatter(
            x=non_ref_df["Stack Energy Consumption [kWh/kg]"],
            y=non_ref_df["Stack Energy Consumption [kWh/kg]"],
            mode='markers',
            marker=dict(
                size=8,
                color='blue',
                opacity=0.7
            ),
            name='Architectures',
            hovertext=non_ref_df['Electrolyzer Index']
        ))

    # Add a trace for reference architectures with a star symbol
    fig.add_trace(go.Scatter(
        x=ref_df["CAPEX_Random_Number [$1000]"],
        y=ref_df["Stack Energy Consumption [kWh/kg]"],
        mode='markers',
        marker=dict(
            symbol='star',  # Use star symbol
            size=12,        # Adjust size if needed
            color='gold',   # Specify color for star markers
            line=dict(width=1, color='black')  # Outline for better visibility
        ),
        name='Reference Architectures',
        hovertext=ref_df['Electrolyzer Index']
    ))

    # Update layout for the figure
    fig.update_layout(
        title='Energy Consumption per KG vs. CAPEX_Random_Number [$1000]',
        xaxis_title="CAPEX_Random_Number [$1000]",
        yaxis_title="Stack Energy Consumption [kWh/kg]",
        font=dict(size=14),
        autosize=False,
        width=800,
        height=600,
        legend_title_text=color_by if color_by else 'Electrolyzer Type',
    )

    return fig


In [ ]:
output_fig = plot_tradespace(df_cross, color_by=None, filter=None, filter_range=None, ref_architectures=[])
output_fig.show()

KeyError: 'Stack Energy Consumption [kWh]'